# Blind CIV Survey by Burchett et al. [v1.1]

In [1]:
# imports
import pdb

import pyigm
from astropy import constants as const
from astropy import units as u

from linetools.isgm.abscomponent import AbsComponent
from linetools.abund.ions import name_ion
from linetools import utils as ltu

from pyigm.igm.igmsightline import IGMSightline

In [2]:
c_kms = const.c.to('km/s').value

## Read Component Table

In [22]:
blind_file = pyigm.__path__[0]+'/data/sightlines/Blind_CIV/BlindCIV_measurements.dat'
blind_tbl = Table.read(blind_file, format='ascii')
blind_tbl[-5:]

QSO,z_abs,ion,log_N,log_N_err,b_dop,b_dop_err,vel,vel_err,flag
str13,float64,str6,float64,float64,int64,int64,int64,int64,int64
PG1522+101,0.07523,C IV,13.56,0.05,9,2,-6,1,0
PG1522+101,0.07523,H I,13.87,0.02,26,1,-24,1,0
PG1522+101,0.07523,N V,13.19,0.08,10,4,-4,2,0
PG1407+265,0.07227,C IV,13.47,0.04,22,3,-17,2,0
PG1407+265,0.07227,H I,0.0,0.0,9999,9999,0,0,9


In [4]:
flag_dict = {0:1, 1:2, 3:2, 9: 3, -99: 3}

## Read RA/DEC Table

In [20]:
blindradec_file = pyigm.__path__[0]+'/data/sightlines/Blind_CIV/BlindCIV_QSOs_FullCoords.dat'

In [21]:
blindciv_radec = Table.read(blindradec_file, format='ascii')
blindciv_radec[0:3]

QSO,QSOlong,RA,DEC,zQSO,CoordString
str13,str19,float64,float64,float64,str19
J0012-1022,001224.01-102226.5,3.1001,-10.374,0.228,001224.02-102226.40
J0809+4619,080908.13+461925.6,122.2839,46.3238,0.657,080908.14+461925.68
J0826+0742,082633.51+074248.3,126.6396,7.7134,0.311,082633.50+074248.24


## Ingest PG1407+265

In [7]:
pg1407_idx = np.where(blind_tbl['QSO'] == 'PG1407+265')[0]
radec = 'J140923.897+261821.043'
coord = ltu.radec_to_coord(radec)

### Generate components

In [8]:
comps = []
for row in blind_tbl[pg1407_idx]:
    # Calculate redshift
    zabs= row['z_abs'] + (1+row['z_abs'])*row['vel']/c_kms
    # Generate
    comp = AbsComponent(coord, name_ion(row['ion']), zabs, np.sqrt(2)*row['b_dop']*np.array([-1,1])*u.km/u.s,
                       Ntup=(flag_dict[row['flag']], row['log_N'], row['log_N_err']))
    # Parse b-values
    comp.bval = row['b_dop']*u.km/u.s
    comp.sig_bval = row['b_dop_err']*u.km/u.s
    # Append
    comps.append(comp)

### Generate the sightline

In [9]:
pg1407 = IGMSightline(coord, 0.94)
pg1407

<IGMSightline: 14:09:23.897 +26:18:21.043, zem=0.940000>

In [10]:
pg1407._components = comps

In [11]:
pg1407._components

[<AbsComponent: 14:09:23.897 +26:18:21.043, Name=CIV_z0.07221, Zion=(6,4), Ej=0 1 / cm, z=0.0722092, vlim=-31.1127 km / s,31.1127 km / s, logN=13.47, sig_N=0.04, flag_N=1>,
 <AbsComponent: 14:09:23.897 +26:18:21.043, Name=HI_z0.07227, Zion=(1,1), Ej=0 1 / cm, z=0.07227, vlim=-14140.7 km / s,14140.7 km / s, logN=0, sig_N=0, flag_N=3>]

### Write a sightline

In [12]:
pg1407_dict = pg1407.to_dict()

In [13]:
pg1407_dict

{'CreationDate': '2016-Nov-17',
 'DEC': 26.305845277777777,
 'RA': 212.3495708333333,
 u'class': 'IGMSightline',
 u'components': {u'CIV_z0.07221': {'A': None,
   'DEC': 26.305845277777777,
   'Ej': 0.0,
   'Name': u'CIV_z0.07221',
   'RA': 212.3495708333333,
   'Zion': (6, 4),
   u'class': 'AbsComponent',
   'comment': u'',
   'flag_N': 1,
   u'lines': {},
   'logN': 13.47,
   'sig_logN': 0.04,
   'vlim': [-31.112698372208094, 31.112698372208094],
   'zcomp': 0.07220919596869912},
  u'HI_z0.07227': {'A': None,
   'DEC': 26.305845277777777,
   'Ej': 0.0,
   'Name': u'HI_z0.07227',
   'RA': 212.3495708333333,
   'Zion': (1, 1),
   u'class': 'AbsComponent',
   'comment': u'',
   'flag_N': 3,
   u'lines': {},
   'logN': 0.0,
   'sig_logN': 0.0,
   'vlim': [-14140.721410168579, 14140.721410168579],
   'zcomp': 0.07227}},
 'em_type': None,
 'name': u'J140923.897+261821.043',
 'sl_type': u'IGM',
 'user': 'xavier',
 'zem': 0.94}

In [14]:
ltu.savejson('PG1407+265_sl.json', pg1407_dict, easy_to_read=True, overwrite=True)

### Read the sightline

In [13]:
pg1407_dict = ltu.loadjson('PG1407+265_sl.json')

In [15]:
pg1407 = IGMSightline.from_dict(pg1407_dict)
pg1407

> /Users/xavier/local/Python/pyigm/pyigm/igm/igmsightline.py(43)from_dict()
-> 
(Pdb) c


<IGMSightline: 14:09:23.897 +26:18:21.043, zem=0.940000>

In [16]:
pg1407._components

[<AbsComponent: 14:09:23.897 +26:18:21.043, Name=HI_z0.07227, Zion=(1,1), Ej=0 1 / cm, z=0.07227, vlim=-14140.7 km / s,14140.7 km / s, logN=0, sig_N=0, flag_N=3>,
 <AbsComponent: 14:09:23.897 +26:18:21.043, Name=CIV_z0.07221, Zion=(6,4), Ej=0 1 / cm, z=0.0722092, vlim=-31.1127 km / s,31.1127 km / s, logN=13.47, sig_N=0.04, flag_N=1>]

## Full Sample

In [15]:
out_path = pyigm.__path__[0]+'/data/sightlines/Blind_CIV/'

In [23]:
for qso in blindciv_radec:
    # Match to component table
    idx = np.where(blind_tbl['QSO'] == qso['QSO'])[0]
    coord = ltu.radec_to_coord('J'+qso['CoordString'])
    # 
    comps = []
    for row in blind_tbl[idx]:
        # Calculate redshift
        zabs= row['z_abs'] + (1+row['z_abs'])*row['vel']/c_kms
        # Generate
        comp = AbsComponent(coord, name_ion(row['ion']), zabs, np.sqrt(2)*row['b_dop']*np.array([-1,1])*u.km/u.s,
                           Ntup=(flag_dict[row['flag']], row['log_N'], row['log_N_err']))
        # Parse b-values
        comp.bval = row['b_dop']*u.km/u.s
        comp.sig_bval = row['b_dop_err']*u.km/u.s
        # Append
        comps.append(comp)
    # Sightline
    sl = IGMSightline(coord, qso['zQSO'])
    sl._components = comps
    # Write
    sl_dict = sl.to_dict()
    outfile = out_path+'J{:s}.json'.format(qso['CoordString'])
    ltu.savejson(outfile, sl_dict, easy_to_read=True, overwrite=True)
    print("Wrote: {:s}".format(outfile))

Wrote: /Users/xavier/local/Python/pyigm/pyigm/data/sightlines/Blind_CIV/J001224.02-102226.40.json
Wrote: /Users/xavier/local/Python/pyigm/pyigm/data/sightlines/Blind_CIV/J080908.14+461925.68.json
Wrote: /Users/xavier/local/Python/pyigm/pyigm/data/sightlines/Blind_CIV/J082633.50+074248.24.json
Wrote: /Users/xavier/local/Python/pyigm/pyigm/data/sightlines/Blind_CIV/J084349.49+411741.64.json
Wrote: /Users/xavier/local/Python/pyigm/pyigm/data/sightlines/Blind_CIV/J091235.42+295725.56.json
Wrote: /Users/xavier/local/Python/pyigm/pyigm/data/sightlines/Blind_CIV/J092554.43+453544.52.json
Wrote: /Users/xavier/local/Python/pyigm/pyigm/data/sightlines/Blind_CIV/J092909.79+464424.00.json
Wrote: /Users/xavier/local/Python/pyigm/pyigm/data/sightlines/Blind_CIV/J094621.26+471131.20.json
Wrote: /Users/xavier/local/Python/pyigm/pyigm/data/sightlines/Blind_CIV/J094733.22+100508.88.json
Wrote: /Users/xavier/local/Python/pyigm/pyigm/data/sightlines/Blind_CIV/J094952.92+390203.84.json
Wrote: /Users/xavier